In [ ]:
#@title State-Aware ComfyUI Launcher
#@markdown ## 🚀 Launch ComfyUI with Smart Caching
#@markdown This notebook is designed to be state-aware and idempotent.
#@markdown - **On first run:** It will clone the necessary repositories and install dependencies.
#@markdown - **On subsequent runs:** It will quickly update the main repository and skip any steps that have already been completed.

import os
import time
from google.colab import drive

print("✅ [1/5] Initializing...")
start_time = time.time()

# --- 1. Initialization: Mount Google Drive ---
# All data will be stored on your Google Drive for persistence.
print("🔄 [2/5] Mounting Google Drive at /content/drive...")
drive.mount('/content/drive')
print("✅ Google Drive mounted successfully.")

# --- 2. Core Environment Setup (State-Aware Logic) ---
# This section checks if a ComfyUI environment already exists and acts accordingly.
print("\n🔄 [3/5] Setting up Core ComfyUI Environment...")
COMFYUI_OPTIMIZED_PATH = "/content/drive/MyDrive/ComfyUI_Optimized"
COMFYUI_PATH = os.path.join(COMFYUI_OPTIMIZED_PATH, "ComfyUI")

# We create the parent directory if it doesn't exist.
os.makedirs(COMFYUI_OPTIMIZED_PATH, exist_ok=True)

# State Check: Does the ComfyUI repository already exist?
if os.path.exists(COMFYUI_PATH):
    # STATE=TRUE: Environment exists. Update it by pulling the latest changes.
    print(f"  -> Found existing ComfyUI at {COMFYUI_PATH}.")
    print("  -> Updating ComfyUI repository...")
    %cd {COMFYUI_PATH}
    !git pull
    print("  -> ComfyUI is up to date.")
else:
    # STATE=FALSE: Initial setup. Clone the repository into the correct directory.
    print(f"  -> ComfyUI not found. Performing initial clone...")
    %cd {COMFYUI_OPTIMIZED_PATH}
    !git clone https://github.com/comfyanonymous/ComfyUI.git
    print("  -> ComfyUI cloned successfully.")

# --- 3. Dependency Management (Idempotent Actions) ---
# These actions are designed to be safe to run multiple times.
print("\n🔄 [4/5] Installing and Verifying Dependencies...")

# Dependency 1: Custom Node - IPAdapter Plus
# This logic is idempotent: it only clones if the directory is missing.
# It will not re-download or update if the folder already exists.
IPADAPTER_PATH = os.path.join(COMFYUI_PATH, "custom_nodes/ComfyUI_IPAdapter_plus")
print(f"  -> Checking for IPAdapter+ at {IPADAPTER_PATH}...")
if not os.path.exists(IPADAPTER_PATH):
    print("    -> IPAdapter+ not found. Cloning repository...")
    # The clone is directed into the specific target path.
    !git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git {IPADAPTER_PATH}
    print("    -> IPAdapter+ cloned successfully.")
else:
    print("    -> IPAdapter+ directory already exists. Skipping clone.")

# Dependency 2: Python Packages
# pip is naturally idempotent; it won't reinstall packages that are already present and correct.
print("  -> Installing Python packages (piexif and requirements.txt)...")
!pip install piexif -q
%cd {COMFYUI_PATH}
!pip install -r requirements.txt -q
print("  -> Python packages are up to date.")

# --- 4. Execution ---
print("\n🔄 [5/5] Launching ComfyUI...")
print("  -> This may take a few minutes on first launch as models are loaded.")
print("  -> A public link (ending in .gradio.live) will appear below.")

%cd {COMFYUI_PATH}
# --listen is for local network, --share creates a public Gradio link for easy access.
!python main.py --listen --share

# --- End of Execution ---
print("\n" + "="*80)
total_time = time.time() - start_time
print(f"✅ Notebook execution finished. Total time: {total_time:.2f} seconds.")
print("If the launcher did not start, please check the output above for errors.")
print("="*80)